In [2]:
%pip install evaluate nltk rouge_score absl-py


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import evaluate
import json
import os
import math

/Users/vh19/.local/share/mise/installs/python/3.13.5/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Change these variables to perform evalution for you task
original_quiz_path = os.path.abspath(os.getcwd()).split('gpt5_evaluation_scripts')[0] + 'processed_data/gpt5/processed_test.json'
generated_quiz_path = os.path.abspath(os.getcwd()).split('gpt5_evaluation_scripts')[0] + 'generated_data_gpt5/a1_test.json'
print(original_quiz_path)
print(generated_quiz_path)

/Users/vh19/Documents/Works/personal-projects/EduQuiz-vh/processed_data/gpt5/processed_test.json
/Users/vh19/Documents/Works/personal-projects/EduQuiz-vh/generated_data_gpt5/a1_test.json


In [5]:
# Get original quizzes
original_quizzes = []

for line in open(original_quiz_path):
    original_quizzes.append(json.loads(line)['completion'].split("\n###")[0].replace("\n"," ").strip())

# Get generated quizzes
generated_quizzes = []
with open(generated_quiz_path) as f:
    generated_quizzes_dict = json.load(f)

for key in generated_quizzes_dict:
    generated_quizzes.append(generated_quizzes_dict[key].replace("\n"," ").strip())

In [6]:
# Put the predictions and gold references in lists

predictions = []
gold_references = []
# temporary changes to make it work for a1_test.json
#for i in range(len(original_quizzes)):
for i in range(len(generated_quizzes)):
    predictions.append(generated_quizzes[i])
    gold_references.append([original_quizzes[i]])  # BLEU expects list of reference strings

print("Predictions:", predictions)
print("Gold references:", gold_references)

Predictions: ['Question: What made the Christmas Day especially memorable for Alice? True answer: She received a doll presented as a long-belated gift from “Santa.” False answer: She was reunited with her long-lost mother. False answer: She surprised Katie with a trip to see Santa at the mall. False answer: She received an expensive piece of jewelry from her children.', 'Question: What made Alice cry with joy on Christmas Day? True answer: Receiving a long-awaited doll presented as a late gift from “Santa.” False answer: Hearing that her mother was still alive. False answer: Seeing Katie refuse her Christmas gifts. False answer: Finding the message card addressed to Katie.', 'Question: What made the Christmas memorable for Alice in 1982? True answer: Receiving a long-awaited doll presented by “Santa” as if from 1925 False answer: Being reunited with her long-lost mother False answer: Watching Katie open all her presents first False answer: Learning her family could finally afford many 

In [7]:
bleu = evaluate.load('bleu')
rouge = evaluate.load('rouge')

In [8]:
bleu.add_batch(predictions=predictions, references=gold_references)
rouge.add_batch(predictions=predictions, references=gold_references)

In [9]:
final_bleu = bleu.compute()
final_rouge = rouge.compute()

In [13]:
# Define path of downloaded meteor from https://www.cs.cmu.edu/~alavie/METEOR/
meteor_path = os.path.abspath(os.getcwd()).split('gpt5_evaluation_scripts')[0] + '/meteor-1.5/'

# Move results to meteor directory
with open(meteor_path + "predictions.txt", 'w') as f:
    for i in range(len(predictions)):
        f.write(str(predictions[i]) + '\n')
        
with open(meteor_path + "ground_truth.txt", 'w') as f:
    for i in range(len(gold_references)):
        f.write(str(gold_references[i]) + '\n')

# Run the meteor command from the meteor directory and remove result files again   
wd = os.getcwd()
os.chdir(meteor_path)
output = os.popen("java -Xmx2G -jar meteor-*.jar predictions.txt ground_truth.txt -l en -norm").read()
os.remove(meteor_path + "predictions.txt")
os.remove(meteor_path + "ground_truth.txt")
os.chdir(wd)

# Get the score from the output
meteor_score = round(float(output.split("Final score:")[1].strip()) * 100, 2)

In [15]:
print("BLEU: ", str(round(final_bleu['bleu'] * 100, 2)))
print(f"ROUGE-L: {final_rouge['rougeL'] * 100:.2f}")
print("METEOR: ", str(meteor_score))

BLEU:  15.65
ROUGE-L: 26.96
METEOR:  22.68


In [18]:
# Create result files
        
with open(os.path.abspath(os.getcwd()).split('gpt5_evaluation_scripts')[0] + 'generated_data_gpt5/' + 'automatic_evaluation_EEQG.txt', 'w') as f:
    f.write("BLEU: " + str(round(final_bleu['bleu'] * 100, 2)))
    f.write('\n')
    f.write("ROUGE-L: " + str(round(final_rouge['rougeL'] * 100, 2)))
    f.write('\n')
    f.write("METEOR: " + str(meteor_score))

In [24]:
# Get all the target prompts and completions
test_data = []
for line in open(original_quiz_path):
    test_data.append((json.loads(line)))

# Get quizzes
quizzes = []
with open(generated_quiz_path) as f:
    gen_quizzes = json.load(f)

for key in gen_quizzes:
    quizzes.append(gen_quizzes[key])

print(f"Test data length: {len(test_data)}")
print(f"Generated quizzes length: {len(quizzes)}")

# Create human evaluation file - handle small datasets
count = 0
output_dir = os.path.abspath(os.getcwd()).split('gpt5_evaluation_scripts')[0] + 'human_evaluation/EEQG/'
os.makedirs(output_dir, exist_ok=True)  # Create directory if it doesn't exist

with open(output_dir + 'EEQG_gpt5.txt', 'w') as f:
    # Use the smaller of the two datasets to avoid index errors
    max_samples = min(len(test_data), len(quizzes), 100)
    step_size = max(1, len(quizzes) // max_samples) if len(quizzes) > max_samples else 1
    
    for i in range(0, len(quizzes), step_size):
        if count >= max_samples:
            break
            
        f.write('Excel row: ' + str(count+1) + ' Test instance: ' + str(i+1) + '\n\n')
        f.write(test_data[i]['prompt'].split("\n\n###")[0] + '\n\n')
        f.write('Generated quiz:\n')
        f.write(quizzes[i].strip() + '\n\n')
        f.write('----------------------------------------------------------------------------------------' + '\n\n')
        count+=1

print(f"Created human evaluation file with {count} samples")


Test data length: 950
Generated quizzes length: 5
Created human evaluation file with 5 samples
